In [1]:
from vllm import LLM, SamplingParams
from dotenv import load_dotenv

load_dotenv()

/home/abhirath/miniconda3/envs/decoder/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-14 14:53:16,143	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


True

In [2]:
llm = LLM(model="google/gemma-2b", gpu_memory_utilization = 0.98, max_model_len=8000)

string = "Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?"


INFO 07-14 14:53:40 llm_engine.py:169] Initializing an LLM engine (v0.5.1) with config: model='google/gemma-2b', speculative_config=None, tokenizer='google/gemma-2b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=google/gemma-2b, use_v2_block_manager=False, enable_prefix_caching=False)
WARNING 07-14 14:53:44 gemma.py:56] Gemma's activation function was incorrectly set to exact GeLU in the config JSON file when it was initially released. Changing th

In [8]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens = 1024, logprobs=10)
output = llm.generate(string, sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.90s/it, est. speed input: 11.04 toks/s, output: 53.84 toks/s]


In [12]:
output[0].outputs[0].logprobs[0]

{109: Logprob(logprob=-0.06430083513259888, rank=1, decoded_token='\n\n'),
 586: Logprob(logprob=-3.720550775527954, rank=2, decoded_token=' A'),
 1: Logprob(logprob=-4.283051013946533, rank=3, decoded_token=''),
 235248: Logprob(logprob=-4.783051013946533, rank=4, decoded_token=' '),
 591: Logprob(logprob=-5.220551013946533, rank=5, decoded_token=' ('),
 649: Logprob(logprob=-5.220551013946533, rank=6, decoded_token=' *'),
 108: Logprob(logprob=-5.283051013946533, rank=7, decoded_token='\n'),
 3: Logprob(logprob=-inf, rank=8, decoded_token=''),
 2: Logprob(logprob=-inf, rank=9, decoded_token=''),
 0: Logprob(logprob=-inf, rank=10, decoded_token='')}

In [18]:
# We make a decoder class that allows us to examine the model.
import torch

class LMToolkit:
    
    def __init__(self, model, max_new_tokens : int = 1024, topk : int = 10):
        """ 
            Args:
                model: The model is expected to be an instance of a vLLM LLM.
                
        """
        
        self.model = model
        self.tokenizer = self.model.llm_engine.tokenizer.tokenizer
        self.max_new_tokens = max_new_tokens
        self.topk = topk
        
        self.model.llm_engine.model_config.max_logprobs = self.topk + 1
        
    @torch.inference_mode()
    def get_next_topk_tokens(self, input_string : str):
        """
            Args:
                input_string: The input string to the model.
                
            Returns:
                A list of tuples. Each tuple contains a token and its log probability.
        """
        
        sampling_params = SamplingParams(
            n = 1, 
            temperature = 0, 
            top_p=1,
            max_tokens=1,
            logprobs=self.topk    
        )
        
        outputs = self.model.generate(
            input_string, 
            sampling_params, 
            use_tqdm = False
        )[0].outputs[0].logprobs[0]
        
        topk_tokens = {'decoded': [], 'probs': [], 'token_id': [], 'logprobs': []}
        
        for token_id, logprob_obj in outputs.items():
            
            topk_tokens['logprobs'].append({token_id: logprob_obj})
            topk_tokens['decoded'].append(logprob_obj.decoded_token)
            topk_tokens['probs'].append(logprob_obj.logprob)
            topk_tokens['token_id'].append(token_id)

        topk_tokens['probs'] = torch.exp(torch.tensor(topk_tokens['probs'])).tolist()

        return topk_tokens
        
        
        
        
    


In [19]:
new_decoder = LMDecoder(llm)
new_decoder.get_next_topk_tokens(string)

{'decoded': ['\n\n', ' A', '', ' ', ' (', ' *', '\n', ' Write', 'A', ' a'],
 'probs': [0.7080947160720825,
  0.037527646869421005,
  0.024229668080806732,
  0.016652787104249,
  0.011445282027125359,
  0.011445282027125359,
  0.010751848109066486,
  0.010100426152348518,
  0.008373547345399857,
  0.007866219617426395],
 'token_id': [109, 586, 1, 235248, 591, 649, 108, 15615, 235280, 476],
 'logprobs': [{109: Logprob(logprob=-0.34517744183540344, rank=1, decoded_token='\n\n')},
  {586: Logprob(logprob=-3.282677412033081, rank=2, decoded_token=' A')},
  {1: Logprob(logprob=-3.720177412033081, rank=3, decoded_token='')},
  {235248: Logprob(logprob=-4.09517765045166, rank=4, decoded_token=' ')},
  {591: Logprob(logprob=-4.47017765045166, rank=5, decoded_token=' (')},
  {649: Logprob(logprob=-4.47017765045166, rank=6, decoded_token=' *')},
  {108: Logprob(logprob=-4.53267765045166, rank=7, decoded_token='\n')},
  {15615: Logprob(logprob=-4.59517765045166, rank=8, decoded_token=' Write')},
 